In [1]:
# Import necessary libraries
import os
import win32com.client as win32  # For COM connection with Aspen Plus
import numpy as np  # For numerical calculations
import time  # To pause code execution for Aspen run completion
import pandas as pd  # To create and manipulate dataframes
import matplotlib.pyplot as plt  # For creating plots
from SALib.analyze import sobol # For Sobol SA
from SALib.sample import sobol as sobol_sample
from SALib.sample import saltelli
from SALib.test_functions import Ishigami
import warnings # To mute warning
from sklearn.ensemble import RandomForestRegressor

# Muting warnings
warnings.filterwarnings('ignore', category = DeprecationWarning) 
warnings.filterwarnings('ignore', category = FutureWarning)

In [2]:
# Frequency factors using Saltelli sampler
factors = {
    'num_vars': 3,
    'names': ['K1', 'K2', 'K3'],
    'bounds': [[1000, 6000],
               [10000, 60000],
               [100, 600]]
}
frequency_factor_values = sobol_sample.sample(factors, 1024)

In [3]:
# File path to the Aspen simulation
Aspenfilepath = r'Template\Urea_reactor_biuret.bkp'
# Access the Aspen Plus simulation
Aspen = win32.Dispatch('Apwn.document')
Aspen.InitFromArchive2(os.path.abspath(Aspenfilepath))
Aspen.Engine.Run2()
Aspen.Visible = True # Optional
Aspen.SuppressDialogs = 1

In [4]:
# Define the Aspen data nodes for different mass fractions and reaction parameters
k1_node = r'\Data\Reactions\Reactions\R1\Input\PRE_EXP\1'
k2_node = r'\Data\Reactions\Reactions\R1\Input\PRE_EXP\2'
k3_node = r'\Data\Reactions\Reactions\R1\Input\PRE_EXP\3'
urea_mass_fraction_node = r'\Data\Streams\UREAPROD\Output\MASSFRAC\MIXED\UREA'
carb_mass_fraction_node = r'\Data\Streams\UREAPROD\Output\MASSFRAC\MIXED\CARB'
co2_mass_fraction_node = r'\Data\Streams\UREAPROD\Output\MASSFRAC\MIXED\CO2'
nh3_mass_fraction_node = r'\Data\Streams\UREAPROD\Output\MASSFRAC\MIXED\NH3'
h2o_mass_fraction_node = r'\Data\Streams\UREAPROD\Output\MASSFRAC\MIXED\H2O'
biuret_mass_fraction_node = r'\Data\Streams\UREAPROD\Output\MASSFRAC\MIXED\BIURETU'
run_status_node = r'\Data\Results Summary\Run-Status\Output\PER_ERROR\2'


# Initialize lists to store the mass fractions at each K value
urea, carb, co2, nh3, h2o, biuret = [], [], [], [], [], []

# Initialize list to store k values that arise errors within the Aspen Plus simulation
k_errors = []

# Loop through each K value, update the Aspen model, and retrieve mass fractions
for ki in frequency_factor_values:  
    Aspen.Tree.FindNode(k1_node).Value = ki[0]
    Aspen.Tree.FindNode(k2_node).Value = ki[1]
    Aspen.Tree.FindNode(k3_node).Value = ki[2]
    Aspen.Reinit() # Reset the Aspen simulation
    Aspen.Engine.Run2() # Run the Aspen simulation
    
    # Wait until the simulation is finished
    while Aspen.Engine.IsRunning == True:
        time.sleep(0.7)
    
    status_message = Aspen.Tree.FindNode(run_status_node).Value # Use status message to check for convergence
    # Logic to get values that does not include errors
    if 'errors' not in status_message:
        urea.append(Aspen.Tree.FindNode(urea_mass_fraction_node).Value)
        carb.append(Aspen.Tree.FindNode(carb_mass_fraction_node).Value)
        co2.append(Aspen.Tree.FindNode(co2_mass_fraction_node).Value)
        nh3.append(Aspen.Tree.FindNode(nh3_mass_fraction_node).Value)
        h2o.append(Aspen.Tree.FindNode(h2o_mass_fraction_node).Value)
        biuret.append(Aspen.Tree.FindNode(biuret_mass_fraction_node).Value)
    else:
        k_errors.append([ki[0], ki[1], ki[2]]) # Store k values that get errors
        
# Close the Aspen Plus connection
Aspen.Close()

In [45]:
# Store data into excel files
mass_fraction_data = {
    'Urea': urea,
    'Carb': carb,
    'CO2': co2,
    'NH3': nh3,
    'H2O': h2o,
    'Biuret': biuret
}

k_errors_data = {
    'K': k_errors
}

# Creating a DataFrame from the dictionary
df_1 = pd.DataFrame(mass_fraction_data)
df_2 = pd.DataFrame(k_errors_data)

# Saving the DataFrame to an Excel file
df_1.to_excel('massfrac.xlsx')
df_2.to_excel('kerrors.xlsx')

In [5]:
# Delete k sets that arose errors
new_k_errors = []

for i in range(0, len(k_errors)):
    list = float(k_errors[i][0]), float(k_errors[i][1]), float(k_errors[i][2])
    new_k_errors.append(list)

matching_indices = [i for i, arr in enumerate(frequency_factor_values) if any(np.array_equal(arr, check) for check in new_k_errors)]
filtered_k_values = np.delete(frequency_factor_values, matching_indices, 0)

In [6]:
# Function to calculate percentage error between measured and reference values
def Calculate_percentage_error(measured, reference):
    return np.abs((measured - reference) / reference) * 100

# Define reference mass fractions for comparison (urea, carb, CO2, NH3, H2O, biuret)
Reference_mass_frac = np.array([0.322046637078701, 0.265199938619204, 0.0389103609578698, 0.191659325990155, 0.16835012629242, 0.00254959024013388])

# Calculate percentage errors for each component
Error_urea = Calculate_percentage_error(urea, Reference_mass_frac[0])
Error_carb = Calculate_percentage_error(carb, Reference_mass_frac[1])
Error_co2 = Calculate_percentage_error(co2, Reference_mass_frac[2])
Error_nh3 = Calculate_percentage_error(nh3, Reference_mass_frac[3])
Error_h2o = Calculate_percentage_error(h2o, Reference_mass_frac[4])
Error_biuret = Calculate_percentage_error(biuret, Reference_mass_frac[5])

# Function to compute the group percentage error for each frequency factor
def compute_group_percentage_error(Urea, Carb, CO2, NH3, H2O, Biuret, Reference_mass_frac):
    # Initialize a list to store the group percentage error for each frequency factor
    group_errors = []

    # Loop through each frequency factor (row in Urea, Carb, etc.)
    for i in range(len(Urea)):
        # Calculate percentage errors for all species
        error_urea = Calculate_percentage_error(Urea[i], Reference_mass_frac[0])
        error_carb = Calculate_percentage_error(Carb[i], Reference_mass_frac[1])
        error_co2 = Calculate_percentage_error(CO2[i], Reference_mass_frac[2])
        error_nh3 = Calculate_percentage_error(NH3[i], Reference_mass_frac[3])
        error_h2o = Calculate_percentage_error(H2O[i], Reference_mass_frac[4])
        error_biuret = Calculate_percentage_error(Biuret[i], Reference_mass_frac[5])
        
        # Combine the errors into a single metric (e.g., mean of all species' errors)
        mean_error = np.mean([error_urea, error_carb, error_co2, error_nh3, error_h2o, error_biuret])
        
        # Append the group error for this frequency factor
        group_errors.append(mean_error)
    
    return np.array(group_errors)

# Calculate the group percentage error for each frequency factor
group_percentage_errors = compute_group_percentage_error(urea, carb, co2, nh3, h2o, biuret, Reference_mass_frac)


In [7]:
# Find the index of the minimum error
min_error_index = np.argmin(group_percentage_errors)
min_error = group_percentage_errors[min_error_index]

In [16]:
print(f'Urea mass fraction: {urea[min_error_index]:.4f}')
print(f'Carbamate mass fraction: {carb[min_error_index]:.4f}')
print(f'CO2 mass fraction: {co2[min_error_index]:.4f}')
print(f'NH3 mass fraciton: {nh3[min_error_index]:.4f}')
print(f'H2O mass fraciton: {h2o[min_error_index]:.4f}')
print(f'Biuret mass fraciton: {biuret[min_error_index]:.4f}')
print(f'Error: {min_error:.4f}%')
print(f'k1: {filtered_k_values[min_error_index][0]:.2f}')
print(f'k2: {filtered_k_values[min_error_index][1]:.2f}')
print(f'k3: {filtered_k_values[min_error_index][2]:.2f}')

Urea mass fraction: 0.3323
Carbamate mass fraction: 0.2418
CO2 mass fraction: 0.0325
NH3 mass fraciton: 0.2082
H2O mass fraciton: 0.1710
Biuret mass fraciton: 0.0031
Error: 9.7274%
k1: 3437.32
k2: 14125.76
k3: 425.94


In [17]:
# Step 1: Fit a surrogate model
model = RandomForestRegressor()
model.fit(filtered_k_values, group_percentage_errors)

# Function to evaluate the model
def evaluate_model(X_new):
    return model.predict(np.array(X_new).reshape(1, -1))

# Generate samples
param_values = saltelli.sample(factors, 1024)

# Initialize an array for the outputs
Y = np.zeros(param_values.shape[0])

# Evaluate the surrogate model on the sampled inputs
for i, X_sample in enumerate(param_values):
    Y[i] = evaluate_model(X_sample)

# Perform Sobol sensitivity analysis
Si = sobol.analyze(factors, Y)

# Display results
print("Sobol Sensitivity Indices:")
print("First Order Indices:", Si['S1'])
print("Total Order Indices:", Si['ST'])

Sobol Sensitivity Indices:
First Order Indices: [0.4940022  0.21842613 0.17041413]
Total Order Indices: [0.59370564 0.29399177 0.24335776]
